In [1]:
# TÍTULO: Inferencia y Generación de Recortes (ROI)
# OBJETIVO: Validar el modelo de autos y generar los inputs para el detector de placas.

from ultralytics import YOLO
import cv2
import os
import matplotlib.pyplot as plt
import glob
from tqdm.notebook import tqdm

# --- CONFIGURACIÓN ---
# Ruta relativa a tu mejor modelo (ajusta la fecha si es necesario)
MODEL_PATH = '../../production_weights/01_autos_best.pt' 
# Si aún no lo moviste a production, usa la ruta de models/vehicle_detector/...

# Imágenes de prueba (pueden ser las del test set o nuevas descargadas de internet)
TEST_IMAGES_DIR = '../../datasets/01_autos/test/images'
OUTPUT_DIR = '../../data/processed/car_crops_test'

# Cargar Modelo
model = YOLO(MODEL_PATH)

# Crear carpeta de salida
os.makedirs(OUTPUT_DIR, exist_ok=True)

def extract_crops(source_dir, output_dir):
    image_paths = glob.glob(os.path.join(source_dir, '*.jpg'))
    print(f"📸 Procesando {len(image_paths)} imágenes...")
    
    count = 0
    for img_path in tqdm(image_paths):
        # Inferencia
        # conf=0.5: Solo autos con alta confianza
        results = model.predict(img_path, conf=0.5, verbose=False)
        
        original_img = cv2.imread(img_path)
        if original_img is None: continue
        
        for r in results:
            boxes = r.boxes
            for i, box in enumerate(boxes):
                # Coordenadas
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                
                # RECORTE (La magia del pipeline)
                # Validar límites para no romper cv2
                h, w, _ = original_img.shape
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(w, x2), min(h, y2)
                
                crop = original_img[y1:y2, x1:x2]
                
                # Guardar recorte si tiene un tamaño decente
                if crop.size > 0 and crop.shape[0] > 10 and crop.shape[1] > 10:
                    base_name = os.path.splitext(os.path.basename(img_path))[0]
                    save_name = f"{base_name}_crop_{i}.jpg"
                    cv2.imwrite(os.path.join(output_dir, save_name), crop)
                    count += 1

    print(f"✅ ¡Listo! Se generaron {count} recortes de autos en '{output_dir}'")

# Ejecutar
extract_crops(TEST_IMAGES_DIR, OUTPUT_DIR)

📸 Procesando 1615 imágenes...


  0%|          | 0/1615 [00:00<?, ?it/s]

✅ ¡Listo! Se generaron 1647 recortes de autos en '../../data/processed/car_crops_test'


In [2]:
import time

# Mide velocidad pura
start = time.time()
results = model.predict(TEST_IMAGES_DIR, device=0, verbose=False)
end = time.time()

total_time = end - start
fps = len(results) / total_time

print(f"⚡ Velocidad de Inferencia: {fps:.2f} FPS")
print(f"⏱️ Latencia por imagen: {(total_time/len(results))*1000:.2f} ms")

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

⚡ Velocidad de Inferencia: 50.34 FPS
⏱️ Latencia por imagen: 19.86 ms
